In [1]:
'''
2022/04/02 17:27
北大人民医院--瑞舒伐他汀vatin：疗效数据处理
'''

'\n2022/04/02 17:27\n北大人民医院--瑞舒伐他汀vatin：疗效数据处理\n'

# 疗效数据预处理

## 导入程序包

In [2]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np
import datetime

import math
import re
import sys
import os
project_path = os.getcwd()
print(project_path)

D:\PycharmProjects\Rosu_Beijing_renmin_hospital


## 导入预定义函数

In [2]:
# 判断是否为float类型
def judge_float(x):
    try:
        a=float(x)
        return a
    except :
        return np.NaN

In [3]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [4]:
# 过滤特殊字符
def filter_spec_character(x):
    if pd.isnull(x):
        return np.nan
    else:
        x = re.sub('<|>|＞|＜|\+|\/|-|cm|kg','',str(x))
        return x

In [5]:
# 过滤异常大值
def filter_huge_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: np.nan if x<0 else x if float(x) <= 100 * median_value else np.nan)
    df=df[df[feature].notnull()]
    return df

# 读取raw_data

## 用药数据预处理

In [113]:
'''
统计患者京新、可定、其他品牌他汀、其他降脂药的首末次处方时间、用药天数、用药剂量
注意先不需要分冠心病，首先落脚点是在高脂血症，即只要使用了他汀类药物的患者都纳入，当明确诊断为冠心病时则直接归为极高危人群，在后面的分层分析中考虑
'''

'\n统计患者京新、可定、其他品牌他汀、其他降脂药的首末次处方时间、用药天数、用药剂量\n注意先不需要分冠心病，首先落脚点是在高脂血症，即只要使用了他汀类药物的患者都纳入，当明确诊断为冠心病时则直接归为极高危人群，在后面的分层分析中考虑\n'

### 人民医院

In [114]:
# 人民医院用药
df_drug_order_1 = pd.read_excel(r'./data/raw_data/人民医院/4-药物医嘱.xlsx')

In [115]:
# 删除重复用药
df_drug_order_1 = df_drug_order_1.drop_duplicates(df_drug_order_1.columns,keep='first')
# 删除开药总数量为负的，退药记录
df_drug_order_1 = df_drug_order_1[df_drug_order_1['开药总数量']>=0]

In [116]:
# 人民医院--他汀用药
df_drug_vatin_1 = df_drug_order_1[df_drug_order_1.药物通用名称.str.contains('他汀')]

In [117]:
print(df_drug_order_1.shape)
print(df_drug_vatin_1.shape)

(422162, 22)
(251698, 22)


In [118]:
df_drug_vatin_1.columns

Index(['患者编号', '当前就诊次数/命中就诊总次数', '长期或临时', '药物编码', '药物商品名', '药物通用名称', '批准文号',
       '医嘱开始时间', '医嘱停止时间', 'ATC分类码', 'ATC分类名称', 'DDD值', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '用药途径名称', '开药总数量', '开药总数量单位名称', '药物剂型', '下嘱科室名称',
       '数据来源'],
      dtype='object')

In [119]:
# 提取有效字段
df_drug_vatin_1 = df_drug_vatin_1[['患者编号', '药物编码', '药物通用名称','医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型']]

In [120]:
df_drug_vatin_1.shape

(251698, 11)

In [121]:
df_drug_vatin_1.head()

,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,开药总数量单位名称,药物剂型
0,pat_0d9297a346707bce01f0b4637484efe9,瑞舒伐他汀钙,2019-12-31 07:54:51,NaN,旧 10mgx28片/盒,1.0,片,QD(8am),1.0,盒,片剂
1,pat_0ebd83a8c3990a103016a650f84616f8,瑞舒伐他汀钙,2018-03-11 11:23:20,NaN,可定 10mg x7片/盒,10.0,mg,QD(8am),5.0,盒,片剂
2,pat_13da443c402ad02c9e9de0c5c0751eda,阿托伐他汀钙,2018-01-08 16:29:39,NaN,旧 10mgx7片/盒,10.0,mg,QD(8am),5.0,盒,片剂
3,pat_13da443c402ad02c9e9de0c5c0751eda,阿托伐他汀钙,2018-03-22 15:00:58,NaN,旧 10mgx7片/盒,10.0,mg,QD(8am),5.0,盒,片剂
4,pat_13da443c402ad02c9e9de0c5c0751eda,阿托伐他汀钙,2018-06-26 14:08:42,NaN,旧 10mgx7片/盒,10.0,mg,QD(8am),5.0,盒,片剂


In [122]:
df_drug_vatin_1.to_csv(project_path+'/data/pre_processed_raw_data/df_2.3.1_人民_drug_vatin.csv')

### 世纪坛医院

In [123]:
# 世纪坛用药
df_drug_order_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                           sheet_name='用药')

KeyboardInterrupt: 

In [ ]:
# 世纪坛医院-他汀用药
df_drug_order_2 = df_drug_order_2[df_drug_order_2['药品名称'].notnull()]
# 删除重复
df_drug_order_2 = df_drug_order_2.drop_duplicates(df_drug_order_2.columns,keep='first')
# 他汀类药物
df_drug_vatin_2 = df_drug_order_2[df_drug_order_2.药品名称.str.contains('他汀|可定')]

In [ ]:
print(df_drug_order_2.shape)
print(df_drug_vatin_2.shape)

In [ ]:
df_drug_vatin_2.columns

In [ ]:
# 提取有效字段
df_drug_vatin_2 = df_drug_vatin_2[['patient_SN', '就诊时间（门诊）','给药开始时间', '给药停止时间', '用药时长',
                '用药频率', '单次剂量','药品类别', '用药（商品名称）', '药物规格', '药品医嘱内容', '药品名称']]

In [ ]:
# 世纪坛医院的用药频率和药品类别部分数据混乱
# 就诊时间（门诊）-补充-给药开始时间缺失值
df_drug_vatin_2=df_drug_vatin_2.reset_index(drop=True)
for i in range(df_drug_vatin_2.shape[0]):
    x =  df_drug_vatin_2.loc[i,'药品类别']
    if re.match('.*?次', x):
        df_drug_vatin_2.loc[i,'用药频率'] = x
    t = df_drug_vatin_2.loc[i,'就诊时间（门诊）']
    if pd.notnull(t):
        df_drug_vatin_2.loc[i,'给药开始时间'] = t

In [ ]:
df_drug_vatin_2.columns

In [ ]:
# 有效字段排序
df_drug_vatin_2 = df_drug_vatin_2[['patient_SN', '药品名称', '给药开始时间', '给药停止时间', '用药时长', '药物规格', '单次剂量', '用药频率']]
# 修改列名
df_drug_vatin_2.columns = ['患者编号', '药物通用名称','医嘱开始时间', '医嘱停止时间', '用药时长', '药物规格', '药物剂量', '频率名称']

In [ ]:
# 删除医嘱开始时间为空的数据
df_drug_vatin_2 = df_drug_vatin_2[df_drug_vatin_2.医嘱开始时间.notnull()]

In [ ]:
df_drug_vatin_2.shape

In [ ]:
df_drug_vatin_2.head()

In [ ]:
df_drug_vatin_2.to_csv(project_path+'/data/pre_processed_raw_data/df_2.3.2_世纪坛_drug_vatin.csv')

### 合并用药数据

In [ ]:
# 添加数据来源
df_drug_vatin_1.insert(0,'data_source','人民医院')
df_drug_vatin_2.insert(0,'data_source','世纪坛医院')

In [ ]:
# 合并人民医院和世纪坛医院的他汀类用药
df_drug_vatin = pd.concat([df_drug_vatin_1, df_drug_vatin_2],axis=0).reset_index(drop=True)

In [ ]:
df_drug_vatin.shape

In [ ]:
df_drug_vatin.data_source.value_counts()

In [ ]:
df_drug_vatin.head()

In [ ]:
df_drug_vatin.to_csv(project_path+'/data/pre_processed_raw_data/df_2.3.3_他汀用药.csv')

In [72]:
df_drug_vatin=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.3.3_他汀用药.csv')
if 'Unnamed: 0' in df_drug_vatin.columns:
    del df_drug_vatin['Unnamed: 0']

In [73]:
df_drug_vatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长'],
      dtype='object')

### 提取瑞舒伐他汀

In [ ]:
# 提取瑞舒伐他汀
df_drug_resuvatin = df_drug_vatin[df_drug_vatin.药物通用名称.str.contains('瑞舒|可定')]

In [ ]:
df_drug_resuvatin.columns

In [ ]:
# 用药时长和药物规格部分数据混乱
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
# 药物规格drug specifications
for i in range(df_drug_resuvatin.shape[0]):
    s = df_drug_resuvatin.loc[i,'用药时长']
    if re.match('.*?[盒片]', str(s)):
        df_drug_resuvatin.loc[i,'药物规格'] = df_drug_resuvatin.loc[i,'药物规格'] + s

In [ ]:
df_drug_resuvatin.shape

In [ ]:
# 删除重复
df_drug_resuvatin = df_drug_resuvatin.drop_duplicates(df_drug_resuvatin.columns,keep='first')

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.to_csv(project_path + '/data/processed_data/df_2.3.4_瑞舒伐他汀用药.csv')

In [ ]:
# 其它他汀药物
df_drug_vatin_other = df_drug_vatin[~ df_drug_vatin.药物通用名称.str.contains('瑞舒|可定')]
# 删除重复数据
df_drug_vatin_other = df_drug_vatin_other.drop_duplicates(df_drug_vatin_other.columns, keep='first')

In [ ]:
df_drug_vatin_other.shape

In [ ]:
df_drug_vatin_other.to_csv(project_path + '/data/processed_data/df_2.3.4_其它他汀用药.csv')

In [ ]:
df_drug_vatin_other.head(1)

### 打标签：京新、可定

In [ ]:
'''
进口瑞舒伐他汀钙（可定）药物编码为203238，包括3个规格：10mg/片、可定10mg/片、可定10mg×7片/盒。
国产瑞舒伐他汀钙（京新）药物编码为204979，包括4个规格：10mg×28片/盒、京新10mg×28片/盒、旧 10mg×28片/盒、京新10mg/片。
'''

In [ ]:
df_drug_resuvatin.药物规格.value_counts()

In [ ]:
df_drug_resuvatin.head(1)

In [ ]:
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    s = df_drug_resuvatin.loc[i,'药物规格']
    if re.match('京新', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '京新'
    elif re.match('可定', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '可定'
    elif re.match('.*?28片', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '京新'
    elif re.match('.*?7片', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '可定'
    elif re.match('10毫克 /片',s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '可定'

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.3.5_打厂商标签.csv')

In [283]:
df_drug_resuvatin=pd.read_csv(project_path+'/data/processed_data/df_2.3.5_打厂商标签.csv')
if 'Unnamed: 0' in df_drug_resuvatin.columns:
    del df_drug_resuvatin['Unnamed: 0']

### 计算日剂量

#### 转换频率

In [284]:
df_drug_resuvatin.频率名称.unique()

array(['QD(8am)', '1/晚(10PM)', 'QN每晚 (8pm)', 'BID(8-4)', 'ONCE',
       'TID(8-12-4)', 'Q12H (8-8)', 'QD1(8am)', 'QOD5 （8pm）', 'QOD(8am)',
       'QD3(12N)', 'QD6(4pm)', 'QOD1(12N)', 'BID1 (8-4)口服', '必要时',
       'QD4 (5pm)', '05 每晚一次', '01 每日一次', '03 每日三次', '02 每日两次'],
      dtype=object)

In [285]:
# 清洗频次
half=['QOD5 （8pm）','QOD(8am)','QOD1(12N)',]
one=['QD(8am)','1/晚(10PM)', 'QN每晚 (8pm)','QD1(8am)','ONCE','01 每日一次','QD3(12N)', 'QD6(4pm)','QD4 (5pm)']
two=['BID(8-4)','Q12H (8-8)','02 每日两次','BID1 (8-4)口服',]
three=['TID(8-12-4)','03 每日三次',]

df_drug_resuvatin['频率名称']=df_drug_resuvatin['频率名称'].apply(lambda x: 0.5 if x in half else
                                                                          1 if x in one else
                                                                          2 if x in two else
                                                                          3 if x in three else np.nan)

In [286]:
df_drug_resuvatin.频率名称.value_counts()

1.0    56541
2.0      891
3.0       39
0.5       26
Name: 频率名称, dtype: int64

In [287]:
# 删除频率为空的用药
df_drug_resuvatin = df_drug_resuvatin[df_drug_resuvatin.频率名称.notnull()]

In [288]:
df_drug_resuvatin.shape

(57497, 14)

#### 单次剂量处理

In [ ]:
'''
与开药剂量相同的单次剂量: 关于28片的单次剂量修正.xlsx
'''

In [290]:
df_dosage_modify= pd.read_excel(project_path+'/data/raw_data/关于28片的单次剂量修正.xlsx')

In [291]:
df_dosage_modify.head()

,患者编号,单次剂量
0,pat_0c3ca7d7ae58a06efa759f9b79c72fd8,10mg
1,pat_0e7deb0c603dc9c2efc3f37bcd07a05b,10mg
2,pat_1be546098ec4c6d8b317712f6b14f519,20mg
3,pat_2eb269335e17d1b8f473dfb566ad5170,10mg
4,pat_39aab159adbbf11fe0871cf55cecf4dd,10mg


In [ ]:
df_drug_resuvatin.药物剂量单位名称.value_counts()

In [ ]:
# 片--10mg；盒--280mg
df_drug_resuvatin= df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    u = df_drug_resuvatin.loc[i,'药物剂量单位名称']
    # 将数值为1的剂量统一换成10mg
    if df_drug_resuvatin.loc[i,'药物剂量']== 1:
        df_drug_resuvatin.loc[i,'药物剂量']=10
    elif df_drug_resuvatin.loc[i,'药物剂量']==14:
        df_drug_resuvatin.loc[i,'药物剂量']=10
    elif df_drug_resuvatin.loc[i,'药物剂量']== 28:
        df_drug_resuvatin.loc[i,'药物剂量']=10
    elif pd.isnull(u):
        continue
    elif re.match('片',u):
        df_drug_resuvatin.loc[i,'药物剂量']= df_drug_resuvatin.loc[i,'药物剂量']*10

In [ ]:
# 将超过50mg的单次剂量重置为10mg
df_drug_resuvatin['药物剂量'] = df_drug_resuvatin['药物剂量'].apply(lambda x: 10 if x>50 else x)

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.药物剂量.value_counts()

In [ ]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.3.6.2_单次剂量处理.csv')

#### 计算日剂量

In [249]:
df_drug_resuvatin['日剂量']=df_drug_resuvatin['药物剂量'].astype('float') * df_drug_resuvatin['频率名称'].astype('float')

In [ ]:
# 限定瑞舒伐他汀最大日剂量，将超过50mg的日剂量重置为单次剂量
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    if df_drug_resuvatin.loc[i,'日剂量'] > 50:
        df_drug_resuvatin.loc[i,'日剂量']=df_drug_resuvatin.loc[i,'药物剂量']

In [ ]:
df_drug_resuvatin['日剂量'].value_counts()

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.head(1)

In [ ]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.3.6.3_计算日计量.csv')

### 删除重复用药

In [7]:
'''
重复数据、多次开药，取最后一次开药记录
注意，别删掉不同brand的用药，后面需要纳排
'''

'\n重复数据、多次开药，取最后一次开药记录\n注意，别删掉不同brand的用药，后面需要纳排\n'

In [12]:
df_drug_resuvatin=pd.read_csv(project_path+'/data/processed_data/df_2.3.6.3_计算日计量.csv')
if 'Unnamed: 0' in df_drug_resuvatin.columns:
    del df_drug_resuvatin['Unnamed: 0']

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df_drug_resuvatin.shape

(57497, 15)

In [14]:
df_drug_resuvatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量'],
      dtype='object')

In [15]:
# 同一天多次用药记录，取最后一次;
df_drug_resuvatin['医嘱开始日期'] = df_drug_resuvatin.医嘱开始时间.apply(lambda x: str(x).split(' ')[0])
df_drug_resuvatin = df_drug_resuvatin.sort_values(['患者编号','医嘱开始时间'],ascending=True)
df_drug_resuvatin = df_drug_resuvatin.drop_duplicates(subset=['患者编号','resuvatin_brand','医嘱开始日期'],keep='last')

In [16]:
df_drug_resuvatin.shape

(57301, 16)

In [13]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.7_删除用药重复记录.csv')

### 计算用药时长和开药总剂量

In [18]:
'''
人民医院：用药时长 = 开药总剂量/日剂量；医嘱结束时间 = 医嘱开始时间 + 用药时长
世纪坛医院：用药时长 = 用药停止时间-用药开始时间 + 1
'''
'''
人民医院：开药总剂量=开药总数量*盒数
世纪坛医院：开药总剂量=日剂量*用药时长
'''

'\n人民医院：开药总剂量=开药总数量*盒数\n世纪坛医院：开药总剂量=日剂量*用药时长\n'

#### 计算世纪坛--用药时长

In [19]:
df_drug_resuvatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', '医嘱开始日期'],
      dtype='object')

In [20]:
df_drug_resuvatin.shape

(57301, 16)

In [34]:
# 对于世纪坛医院用药结束时间为空的数据，将下一次开药时间作为其上一次结束时间。
temp_list=[]
for i in df_drug_resuvatin.患者编号.unique():
    temp = df_drug_resuvatin[df_drug_resuvatin['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    hospital = temp.loc[0,'data_source']
    if re.match('世纪坛医院',hospital) and (temp.shape[0]>1):
        for j in range(temp.shape[0]):
            # 医嘱结束时间为空
            if pd.isnull(temp.loc[j,'医嘱停止时间']) and (j+1<temp.shape[0]):
                # 提取下一次用药记录
                temp_end = temp[temp['医嘱开始日期'] > temp.loc[j,'医嘱开始日期']].reset_index(drop=True)
                if temp_end.shape[0]>0:
                    temp.loc[j,'医嘱停止时间'] = temp_end.loc[0,'医嘱开始时间']
    temp_list.append(temp)

In [36]:
df_drug_resuvatin=pd.concat(temp_list,axis=0)
del temp_list

In [37]:
df_drug_resuvatin.shape

(57301, 16)

In [38]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_temp_补充用药结束时间.csv')

In [39]:
# 用药时长=医嘱结束时间-医嘱开始时间+1
df_drug_resuvatin['用药时长']=np.nan
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    # 根据医嘱时间计算
    drug_start_time = df_drug_resuvatin.loc[i,'医嘱开始时间']
    drug_end_time = df_drug_resuvatin.loc[i,'医嘱停止时间']
    hospital= df_drug_resuvatin.loc[i,'data_source']
    if re.match('世纪坛医院', hospital):
        df_drug_resuvatin.loc[i,'用药时长'] = (pd.to_datetime(drug_end_time)-pd.to_datetime(drug_start_time)).days + 1
#     # 根据开药剂量计算
#     prescription_sum = df_drug_resuvatin.loc[i,'开药总剂量']
#     dosage = df_drug_resuvatin.loc[i,'日剂量']
#     if pd.isnull(prescription_sum):
#         prescription_time = np.nan
#     else:
#         prescription_time = math.ceil(prescription_sum / dosage) # 向上取整
#     df_drug_resuvatin.loc[i,'用药时长']= max(-1,drug_time,prescription_time)

In [40]:
df_drug_resuvatin.shape

(57301, 16)

In [41]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.8.1_计算世纪坛_用药时长.csv')

#### 计算开药总剂量

In [42]:
# 盒--280mg；片--10mg
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    prescription_u = df_drug_resuvatin.loc[i,'开药总数量单位名称']
    # 世纪坛医院：日剂量*用药时长
    if pd.isnull(prescription_u):
        df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'用药时长'] * df_drug_resuvatin.loc[i,'日剂量']
    # 人民医院: 开药总剂量
    elif re.match('盒', prescription_u):
        # 京新一盒280mg；可定一盒70mg
        brand = df_drug_resuvatin.loc[i,'resuvatin_brand']
        if re.match('京新', brand):
            df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'开药总数量']*280
        else:
            df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'开药总数量']*70
    elif re.match('片', prescription_u):
        df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'开药总数量']*10

In [43]:
# 删除数值为0的开药总剂量
df_drug_resuvatin = df_drug_resuvatin[df_drug_resuvatin['开药总剂量']>0]

In [44]:
df_drug_resuvatin.shape

(54471, 17)

In [47]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.8.2计算开药总剂量.csv')

PermissionError: [Errno 13] Permission denied: 'D:\\PycharmProjects\\Rosu_Beijing_renmin_hospital/data/processed_data/df_2.1.8.2计算开药总剂量.csv'

#### 计算人民医院医嘱结束时间

In [49]:
'''
人民：医嘱停止时间 = 医嘱开始时间 + 用药时长
'''

'\n人民：医嘱停止时间 = 医嘱开始时间 + 用药时长\n'

In [50]:
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    # 计算人民医院：用药时长
    if df_drug_resuvatin.loc[i,'data_source'] == '人民医院':
        d = round(df_drug_resuvatin.loc[i,'开药总剂量']/df_drug_resuvatin.loc[i,'日剂量'],0)
        df_drug_resuvatin.loc[i,'用药时长']= d
        df_drug_resuvatin.loc[i,'医嘱停止时间'] = pd.to_datetime(df_drug_resuvatin.loc[i,'医嘱开始时间']) + datetime.timedelta(days=d)

In [51]:
df_drug_resuvatin.shape

(54471, 17)

In [52]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.8.3_计算人民医院_医嘱结束时间.csv')

## 诊断数据预处理

### 人民医院

In [8]:
# 人民医院--诊断数据
df_diagnose_1 = pd.read_excel(r'./data/raw_data/人民医院/3-诊断.xlsx')
df_diagnose_supplement = pd.read_excel(r'./data/raw_data/人民医院/诊断-补.xlsx')

df_diagnose_1 = pd.concat([df_diagnose_1, df_diagnose_supplement],axis=0).drop_duplicates(subset=df_diagnose_1.columns, keep='first')

In [9]:
df_diagnose_1.shape

(1005384, 8)

In [10]:
df_diagnose_1.columns

Index(['患者编号', '当前就诊次数/命中就诊总次数', '诊断编码', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称',
       '数据来源'],
      dtype='object')

In [11]:
# 提取有效字段
df_diagnose_1 = df_diagnose_1[['患者编号', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称']]
# 删除重复数据
df_diagnose_1 = df_diagnose_1.drop_duplicates(df_diagnose_1.columns, keep='first')

In [14]:
# 患者限定
df_diagnose_1 = df_diagnose_1[df_diagnose_1.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [15]:
df_diagnose_1.shape

(231391, 5)

In [16]:
df_diagnose_1.head()

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
0,pat_0d9297a346707bce01f0b4637484efe9,高胆固醇血症,2019-12-31 07:54:05,否,门诊诊断
1,pat_0ebd83a8c3990a103016a650f84616f8,高胆固醇血症,2018-03-11 11:06:21,否,门诊诊断
121,pat_05eab24ef6eb4e0827cd26e1553a0f64,动脉粥样硬化并高脂血症,2018-02-06 13:20:56,是,门诊诊断
294,pat_04dfcad1038b64914a0cb3dbe835ae57,冠心病,2018-07-20 15:48:42,否,门诊诊断
295,pat_04dfcad1038b64914a0cb3dbe835ae57,高脂血症,2018-07-20 15:48:01,否,门诊诊断


In [ ]:
df_diagnose_1.to_csv(project_path+'/data/pre_processed_raw_data/df_2.4.1_人民医院_诊断.csv')

### 世纪坛医院

In [17]:
# 世纪坛医院--诊断数据
df_diagnose_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                              sheet_name='诊断')

In [18]:
df_diagnose_2.shape

(617655, 8)

In [19]:
df_diagnose_2.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'patient_SN', '就诊次数（住院）', '就诊时间（门诊）',
       '就诊科室（门诊）', 'ICD10编码', 'ICD10名称'],
      dtype='object')

In [20]:
# 提取有效字段
df_diagnose_2 = df_diagnose_2[['patient_SN','ICD10名称','就诊时间（门诊）']]
# 删除重复数据
df_diagnose_2 = df_diagnose_2.drop_duplicates(df_diagnose_2.columns, keep='first')

In [23]:
'患者编号', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称'

('患者编号', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称')

In [24]:
df_diagnose_2.columns = ['患者编号', '诊断名称', '诊断日期']

In [25]:
# 患者限定
df_diagnose_2 = df_diagnose_2[df_diagnose_2.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [26]:
df_diagnose_2.head(1)

,患者编号,诊断名称,诊断日期
234,0020640c4052f1dae97fcf39be390e1b,I25.105,NaT


In [ ]:
df_diagnose_2.to_csv(project_path+'/data/pre_processed_raw_data/df_2.4.2_世纪坛_诊断.csv')

### 合并疾病诊断

In [27]:
# 合并人民医院和世纪坛医院的疾病诊断
df_diagnose = pd.concat([df_diagnose_1, df_diagnose_2],axis=0).reset_index(drop=True)

In [28]:
# 患者限定
df_diagnose = df_diagnose[df_diagnose.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [29]:
df_diagnose.shape

(303522, 5)

In [30]:
df_diagnose.head()

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
0,pat_0d9297a346707bce01f0b4637484efe9,高胆固醇血症,2019-12-31 07:54:05,否,门诊诊断
1,pat_0ebd83a8c3990a103016a650f84616f8,高胆固醇血症,2018-03-11 11:06:21,否,门诊诊断
2,pat_05eab24ef6eb4e0827cd26e1553a0f64,动脉粥样硬化并高脂血症,2018-02-06 13:20:56,是,门诊诊断
3,pat_04dfcad1038b64914a0cb3dbe835ae57,冠心病,2018-07-20 15:48:42,否,门诊诊断
4,pat_04dfcad1038b64914a0cb3dbe835ae57,高脂血症,2018-07-20 15:48:01,否,门诊诊断


In [31]:
df_diagnose.to_csv(project_path+'/data/pre_processed_raw_data/df_2.4.3_诊断.csv')

In [6]:
df_diagnose=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.4.3_诊断.csv')
if 'Unnamed: 0' in df_diagnose.columns:
    del df_diagnose['Unnamed: 0']

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 检验数据预处理

### 人民医院

In [32]:
# 人民医院--检验数据
df_check_1 = pd.read_excel(r'./data/raw_data/人民医院/检验.xlsx',
                          header=1)

In [33]:
df_check_1.shape

(359081, 15)

In [34]:
df_check_1.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', '检验子项结果正常标志',
       '检验子项中文名', '检验子项英文名', '检验子项结果', '检验子项结果数值', '检验子项单位'],
      dtype='object')

In [35]:
df_check_1.columns = ['患者编号', '当前就诊次数/命中就诊总次数', '检验单号', '检验项目名称', '采集时间', '采集时间_date',
       '采集时间_time', '标本名称', '标本类型', '检验子项结果正常标志', '检验子项中文名', '检验子项英文名',
       '检验子项结果', '检验子项结果数值', '检验子项单位']

In [36]:
# 提取有效字段
df_check_1 = df_check_1[['患者编号', '采集时间', '采集时间_date','采集时间_time', '检验子项结果正常标志', '检验子项英文名',
                         '检验子项结果','检验子项结果数值', '检验子项单位']]
# 修改列名
df_check_1 = df_check_1.rename(columns={'采集时间':'检验时间',
                                       '采集时间_date':'检验时间_date',
                                       '采集时间_time':'检验时间_time'})

In [37]:
# 修改项名
df_check_1['检验子项英文名'] = df_check_1['检验子项英文名'].apply(lambda x: 'BUN' if re.match('尿素氮',str(x)) else
                                                                       'Urea' if re.match('尿素',str(x)) else
                                                                       'Glu' if re.match('血糖',str(x)) else
                                                                       'TBIL' if re.match('总胆红素',str(x)) else
                                                                       'TG' if re.match('甘油三酯',str(x)) else
                                                                       'CK' if re.match('肌酸激酶',str(x)) else x)

In [38]:
df_check_1.head(1)

,患者编号,检验时间,检验时间_date,检验时间_time,检验子项结果正常标志,检验子项英文名,检验子项结果,检验子项结果数值,检验子项单位
0,pat_13da443c402ad02c9e9de0c5c0751eda,2018-04-06 07:11:20,2018-04-06,07:11:20,正常,Urea,4.96,4.96,mmol/L


In [39]:
# 删除重复数据
df_check_1 = df_check_1.drop_duplicates(df_check_1.columns, keep='first')
# 删除缺失值
df_check_1 = df_check_1[df_check_1.检验时间_date.notnull()]
df_check_1 = df_check_1[df_check_1['检验子项结果数值'].notnull() & df_check_1.检验子项英文名.notnull()]

In [40]:
df_check_1.shape

(352521, 9)

In [41]:
#删除只有一次的检验
df_check_1_count = df_check_1['患者编号'].value_counts().reset_index()
df_check_1_count = df_check_1_count[df_check_1_count['患者编号']>1]
# 提取剩余的检验
df_check_1 = df_check_1[df_check_1.患者编号.isin(df_check_1_count['index'].unique())]

In [42]:
df_check_1.shape

(352317, 9)

In [43]:
# 限定患者
df_check_1 = df_check_1[df_check_1.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [44]:
df_check_1.shape

(82296, 9)

In [45]:
df_check_1.head()

,患者编号,检验时间,检验时间_date,检验时间_time,检验子项结果正常标志,检验子项英文名,检验子项结果,检验子项结果数值,检验子项单位
261,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,AST,29,29.00,U/L
262,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,CK,67,67.00,U/L
263,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,LDL-C,2.42,2.42,mmol/L
264,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,ALT,28,28.00,U/L
265,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,Urea,5.34,5.34,mmol/L


In [46]:
# 判断是否为float类型
def judge_float(x):
    try:
        a=float(x)
        return a
    except :
        return np.NaN

In [47]:
# 转置表头
df_check_1 = df_check_1.reset_index(drop=True)
df_check_1_T = pd.DataFrame()
count = 0
for i in df_check_1.患者编号.unique():
    count += 1
    # 同一个病人ID的相同检验项
    temp = df_check_1[df_check_1['患者编号']== i].sort_values(['检验时间'],ascending=True)
    temp = temp.drop_duplicates(subset=['患者编号','检验时间_date','检验子项英文名'],keep='last').reset_index(drop=True)
    ## 检验项转置为表头
    if temp.shape[0]>0:
        temp['检验子项结果']=temp['检验子项结果'].apply(judge_float)
        temp= temp.pivot_table('检验子项结果', ['患者编号','检验时间_date'], '检验子项英文名').reset_index()
        df_check_1_T = pd.concat([df_check_1_T,temp],axis=0)

In [48]:
df_check_1_T.shape

(9890, 16)

In [49]:
df_check_1_T.head()

,患者编号,检验时间_date,ALT,AST,CHO,CK,HDL-C,LDL-C,TBIL,TG,Urea,apo-A1,apo-B,CK-MB,BUN,Glu
0,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16,28.0,29.0,3.87,67.0,1.10,2.42,8.5,2.89,5.34,119.5,86.4,NaN,NaN,NaN
1,pat_2bb0dfeccae4278dbf399d6428a2573d,2019-04-10,17.0,16.0,2.82,72.0,0.95,1.43,6.4,1.78,6.21,112.9,60.7,NaN,NaN,NaN
2,pat_2bb0dfeccae4278dbf399d6428a2573d,2019-11-06,15.0,14.0,3.19,57.0,1.06,1.78,8.8,2.56,4.32,117.5,66.8,NaN,NaN,NaN
0,pat_043aba443d80d048105d8903d2034364,2018-06-05,30.0,29.0,4.78,114.0,1.30,3.28,14.8,1.48,6.67,NaN,NaN,NaN,NaN,NaN
0,pat_272f3f5730bfc5a361c461b4fde1dbb8,2018-08-15,16.0,24.0,4.25,133.0,1.24,2.22,11.8,1.15,8.29,125.7,79.2,NaN,NaN,NaN


In [ ]:
df_check_1_T.to_csv(project_path+'/data/pre_processed_raw_data/df_2.5.1_人民_检验.csv')

In [7]:
df_check_1_T=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.5.1_人民_检验.csv')
if 'Unnamed: 0' in df_check_1_T.columns:
    del df_check_1_T['Unnamed: 0']

### 世纪坛医院

In [50]:
# 世纪坛医院--检验数据
df_check_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                           sheet_name='检验')
df_check_2 = df_check_2[:-2]
df_check_2 = df_check_2.dropna(how='all',axis=1)

In [51]:
df_check_2.shape

(36126, 45)

In [52]:
df_check_2.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'patient_SN', '就诊次数（住院）', '就诊时间（门诊）',
       '就诊科室（门诊）', '检验时间', '总胆固醇(TC)-静脉血', '天门冬氨酸氨基转移酶(AST)-静脉血',
       '丙氨酸氨基转移酶(ALT)-静脉血', '总胆红素(TBIL)-静脉血', '肌酐(Crea)-静脉血', '甘油三酯(TG)-静脉血',
       '尿素(Urea)-静脉血(空腹)', '尿素(Urea)-静脉血(空腹)-定性', '尿素(Urea)-静脉血',
       '肌酸激酶(CK)-静脉血(空腹)', '肌酸激酶(CK)-静脉血', '肌酸激酶(CK)-静脉血-定性', '葡萄糖(Glu)-末梢血',
       '葡萄糖(Glu)-静脉血(2小时)', '甘油三酯(TG)-静脉血(空腹)', '甘油三酯(TG)-静脉血(空腹)-定性',
       '甘油三酯(TG)-静脉血-定性', '总胆固醇(TC)-静脉血(空腹)', '总胆固醇(TC)-静脉血(空腹)-定性',
       '载脂蛋白B(apoB)-静脉血(空腹)', '载脂蛋白AI(apoAI)-静脉血(空腹)',
       '载脂蛋白B(apoB)-静脉血(空腹)-定性', '高密度脂蛋白胆固醇(HDL-C)-静脉血(空腹)',
       '低密度脂蛋白胆固醇(LDL-C)-静脉血(空腹)', '低密度脂蛋白胆固醇(LDL-C)-静脉血',
       '高密度脂蛋白胆固醇(HDL-C)-静脉血', '载脂蛋白AI(apoAI)-静脉血', '载脂蛋白B(apoB)-静脉血',
       '脂蛋白a(LPa)-静脉血', '脂蛋白a(LPa)-静脉血-定性', '尿蛋白定性-24小时尿液-定性', '微量总蛋白-24小时尿液',
       '总蛋白(TP)-尿液-定性', '总蛋白(TP)-尿液', '微量白蛋白(mAlb)-尿液', '葡萄糖(Glu)-尿液-定性',
       '肌酸激酶(CK)-静脉血(空腹)-定性', 'Unnamed: 67'],
      dtype='object')

In [53]:
# 删除无效列
df_check_2 = df_check_2.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 67','就诊次数（住院）',
                              '就诊时间（门诊）', '就诊科室（门诊）'],axis=1)
# 删除定性列
q_list = [i for i in df_check_2.columns if '定性' in i]
df_check_2 = df_check_2.drop(q_list,axis=1)
# 修改列名
df_check_2 = df_check_2.rename(columns={'patient_SN':'患者编号',
                                       '检验时间':'检验时间_date'})
df_check_2['检验时间_date'] = df_check_2['检验时间_date'].apply(lambda x: str(x).split(' ')[0])

In [54]:
# 删除重复列
df_check_2 = df_check_2.drop_duplicates(df_check_2.columns, keep='first')
# 删除只有一次的检验
df_check_2_count = df_check_2['患者编号'].value_counts().reset_index()
df_check_2_count = df_check_2_count[df_check_2_count['患者编号']>1]
# 提取剩余的检验
df_check_2 = df_check_2[df_check_2.患者编号.isin(df_check_2_count['index'].unique())]

In [55]:
df_check_2.shape

(32715, 28)

In [56]:
df_check_2.head()

,患者编号,检验时间_date,总胆固醇(TC)-静脉血,天门冬氨酸氨基转移酶(AST)-静脉血,丙氨酸氨基转移酶(ALT)-静脉血,总胆红素(TBIL)-静脉血,肌酐(Crea)-静脉血,甘油三酯(TG)-静脉血,尿素(Urea)-静脉血(空腹),尿素(Urea)-静脉血,...,高密度脂蛋白胆固醇(HDL-C)-静脉血(空腹),低密度脂蛋白胆固醇(LDL-C)-静脉血(空腹),低密度脂蛋白胆固醇(LDL-C)-静脉血,高密度脂蛋白胆固醇(HDL-C)-静脉血,载脂蛋白AI(apoAI)-静脉血,载脂蛋白B(apoB)-静脉血,脂蛋白a(LPa)-静脉血,微量总蛋白-24小时尿液,总蛋白(TP)-尿液,微量白蛋白(mAlb)-尿液
2,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-18,NaN,NaN,NaN,NaN,NaN,NaN,13.41,NaN,...,1.17,1.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-22,NaN,NaN,NaN,NaN,NaN,NaN,11.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-27,NaN,NaN,NaN,NaN,NaN,NaN,9.29,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_check_2.to_csv(project_path+'/data/pre_processed_raw_data/df_2.5.2_世纪坛_检验.csv')

In [8]:
df_check_2=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.5.2_世纪坛_检验.csv')
if 'Unnamed: 0' in df_check_2.columns:
    del df_check_2['Unnamed: 0']

# 数据处理

In [292]:
'''
人口统计学特征、现病史和既往病史、药物使用情况、临床症状／体征、实验室检查和功能检查、患者治疗满意度
'''

'\n人口统计学特征、现病史和既往病史、药物使用情况、临床症状／体征、实验室检查和功能检查、患者治疗满意度\n'

In [293]:
df_drug_resuvatin = pd.read_csv(project_path+'/data/processed_data/df_2.1.8.3_计算人民医院_医嘱结束时间.csv')
if 'Unnamed: 0' in df_drug_resuvatin.columns:
    del df_drug_resuvatin['Unnamed: 0']

In [294]:
df_drug_resuvatin.shape

(54471, 17)

## 纳入高血脂、冠心病患者

In [295]:
df_drug_resuvatin.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,开药总数量单位名称,药物剂型,用药时长,resuvatin_brand,日剂量,医嘱开始日期,开药总剂量
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,NaN,NaN,5.0,京新,10.0,2019-04-04,50.0


In [296]:
# 高脂血症、冠心病
df_diagnose_disease = df_diagnose[df_diagnose['诊断名称'].str.contains('高脂血症|冠心病|冠状动脉粥样硬化性心脏病')]
# 删除重复诊断
df_diagnose_disease = df_diagnose_disease.drop_duplicates(['患者编号','诊断名称'],keep='first')

In [297]:
print(df_diagnose_disease.shape)
print(df_diagnose_disease.患者编号.nunique())

(18322, 5)
9598


In [298]:
df_diagnose_disease.head(1)

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
2,pat_05eab24ef6eb4e0827cd26e1553a0f64,动脉粥样硬化并高脂血症,2018-02-06 13:20:56,是,门诊诊断


In [299]:
df_diagnose_disease.to_csv(project_path+'/data/processed_data/df_temp_冠心病和高脂血病诊断.csv')

In [300]:
# # 合并同一患者的不同诊断
# df_diagnose_disease = df_diagnose_disease.reset_index(drop=True)
# temp_list = []
# for i in df_diagnose_disease.患者编号.unique():
#     temp = df_diagnose_disease[df_diagnose_disease['患者编号']==i][['患者编号','诊断名称']]
#     if temp.shape[0]>1:
#         temp=temp.reset_index(drop=True)
#         temp_1=pd.DataFrame(data={'患者编号':[temp.loc[0,'患者编号']],
#                              '诊断名称':[';'.join(temp['诊断名称'])]})
#         temp_list.append(temp)
#     else:
#         temp_list.append(temp)
# df_diagnose_disease=pd.concat(temp_list,axis=0)
# del temp_list

In [301]:
df_drug_resuvatin.shape

(54471, 17)

In [302]:
# 纳入高脂血症、冠心病
df_drug_resuvatin_disease = df_drug_resuvatin[df_drug_resuvatin.患者编号.isin(df_diagnose_disease.患者编号.unique())]
# 删除重复数据
df_drug_resuvatin_disease = df_drug_resuvatin_disease.drop_duplicates(df_drug_resuvatin_disease.columns,keep='first')

In [303]:
print(df_drug_resuvatin_disease.shape)
print(df_drug_resuvatin_disease.患者编号.nunique())

(51776, 17)
9598


In [304]:
df_drug_resuvatin_disease.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,开药总数量单位名称,药物剂型,用药时长,resuvatin_brand,日剂量,医嘱开始日期,开药总剂量
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,NaN,NaN,5.0,京新,10.0,2019-04-04,50.0


### 冠心病标签

In [305]:
# 提取冠心病
df_disease_CHD = df_diagnose_disease[df_diagnose_disease.诊断名称.str.contains('冠心病|冠状动脉粥样硬化性心脏病')]
df_drug_resuvatin_disease = pd.merge(df_drug_resuvatin_disease, df_disease_CHD[['患者编号','诊断名称']], how='left', on='患者编号')

In [306]:
# 打冠心病标签
df_drug_resuvatin_disease = df_drug_resuvatin_disease.rename(columns={'诊断名称':'冠心病'})
df_drug_resuvatin_disease['冠心病'] = df_drug_resuvatin_disease['冠心病'].apply(lambda x: 0 if pd.isnull(x) else 1)
# 删除相同诊断造成的重复
df_drug_resuvatin_disease =df_drug_resuvatin_disease.drop_duplicates(df_drug_resuvatin_disease.columns,keep='first')

In [307]:
df_drug_resuvatin_disease.冠心病.value_counts()

1    32904
0    18872
Name: 冠心病, dtype: int64

In [308]:
df_drug_resuvatin_disease.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,开药总数量单位名称,药物剂型,用药时长,resuvatin_brand,日剂量,医嘱开始日期,开药总剂量,冠心病
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,NaN,NaN,5.0,京新,10.0,2019-04-04,50.0,1


### 高脂血病标签

In [309]:
# 打高脂血症标签
df_disease_HLP = df_diagnose_disease[df_diagnose_disease.诊断名称.str.contains('高脂血症')]
df_drug_resuvatin_disease = pd.merge(df_drug_resuvatin_disease, df_disease_HLP[['患者编号','诊断名称']], how='left', on='患者编号') 

In [310]:
df_drug_resuvatin_disease = df_drug_resuvatin_disease.rename(columns={'诊断名称':'高脂血症'})
df_drug_resuvatin_disease['高脂血症'] = df_drug_resuvatin_disease['高脂血症'].apply(lambda x: 0 if pd.isnull(x) else 1)
# 删除相同诊断造成的重复
df_drug_resuvatin_disease =df_drug_resuvatin_disease.drop_duplicates(df_drug_resuvatin_disease.columns,keep='first')

In [311]:
df_drug_resuvatin_disease.高脂血症.value_counts()

1    50585
0     1191
Name: 高脂血症, dtype: int64

In [312]:
df_drug_resuvatin_disease.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,开药总数量单位名称,药物剂型,用药时长,resuvatin_brand,日剂量,医嘱开始日期,开药总剂量,冠心病,高脂血症
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,NaN,NaN,5.0,京新,10.0,2019-04-04,50.0,1,1


### 冠心病&高脂血病-标签

In [313]:
# 同时发生冠心病和高血脂病
df_drug_resuvatin_disease['冠心病_高脂血症']=df_drug_resuvatin_disease.apply(lambda x: 1 if (x['冠心病']==1) & (x['高脂血症']==1) 
                                                                      else 0,axis=1)

In [314]:
df_drug_resuvatin_disease.shape

(51776, 20)

In [315]:
df_drug_resuvatin_disease.to_csv(project_path+'/data/processed_data/df_3.1.3_打冠心病和高脂血病标签.csv')

## 排除癌症患者

In [316]:
# 癌症患者
df_diagnose_cancer = df_diagnose[df_diagnose['诊断名称'].str.contains('癌|恶性肿瘤')]
# 排除癌症患者
df_drug_resuvatin_cancer = df_drug_resuvatin_disease[~ df_drug_resuvatin_disease.患者编号.isin(df_diagnose_cancer.患者编号.unique())]

In [317]:
df_drug_resuvatin_cancer.shape

(51151, 20)

In [318]:
df_drug_resuvatin_cancer.to_csv(project_path+'/data/processed_data/df_3.2_排除癌症患者.csv')

In [74]:
df_drug_resuvatin_cancer=pd.read_csv(project_path+'/data/processed_data/df_3.2_排除癌症患者.csv')
if 'Unnamed: 0' in df_drug_resuvatin_cancer.columns:
    del df_drug_resuvatin_cancer['Unnamed: 0']

## 用药数据纳排

In [75]:
'''
基线期：未处方过他汀类药物
研究期：仅京新、可定，未换药
平均日剂量：5-20mg
药物持有率：>=50%
'''

'\n基线期：未处方过他汀类药物\n研究期：仅京新、可定，未换药\n平均日剂量：5-20mg\n药物持有率：>=50%\n'

### 基线期-研究期未换药

In [76]:
'''
基线期(前3月)：未处方过他汀类药物
研究期(1.5-4.5月)：仅京新、可定，未换药
'''

'\n基线期(前3月)：未处方过他汀类药物\n研究期(1.5-4.5月)：仅京新、可定，未换药\n'

In [77]:
df_drug_vatin_other=pd.read_csv(project_path + '/data/processed_data/df_2.3.4_其它他汀用药.csv')
if 'Unnamed: 0' in df_drug_vatin_other.columns:
    del df_drug_vatin_other['Unnamed: 0']

In [78]:
df_drug_vatin_other.shape

(275612, 13)

In [79]:
df_drug_vatin_other.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长'],
      dtype='object')

In [80]:
df_drug_resuvatin_cancer.患者编号.nunique()

9498

In [87]:
temp_list=[]
count=0
for i in df_drug_resuvatin_cancer.患者编号.unique():
    count+=1
    print(count)
    temp = df_drug_resuvatin_cancer[df_drug_resuvatin_cancer['患者编号']==i]
    temp = temp.sort_values(by=['患者编号','医嘱开始时间'],ascending=True).reset_index(drop=True)
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])
    temp['首次用药时间']=drug_start_time
    # 判断研究期是否未换药
    temp_research_period = temp[(pd.to_datetime(temp['医嘱开始时间'])<=drug_start_time+datetime.timedelta(days=135)) &
                                (pd.to_datetime(temp['医嘱开始时间'])>=drug_start_time)]
    if temp_research_period.resuvatin_brand.nunique() >1:
        continue
    # 判断基线期-研究期是否使用其他他汀类用药
    temp_vatin_other = df_drug_vatin_other[df_drug_vatin_other['患者编号']==i]
    baseline_vatin_other = temp_vatin_other[(pd.to_datetime(temp_vatin_other['医嘱开始时间'])<=drug_start_time+datetime.timedelta(days=135)) &
                   (pd.to_datetime(temp_vatin_other['医嘱停止时间'])>=drug_start_time-datetime.timedelta(days=90))]
    if baseline_vatin_other.shape[0]<1:
        temp_list.append(temp)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [ ]:
drug_start_time

In [85]:
temp_baseline

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,...,药物剂型,用药时长,resuvatin_brand,日剂量,医嘱开始日期,开药总剂量,冠心病,高脂血症,冠心病_高脂血症,首次用药时间
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,...,NaN,5.0,京新,10.0,2019-04-04,50.0,1,1,1,2019-04-04 15:35:28


In [ ]:
temp_list

In [83]:
df_drug_resuvatin_cancer=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

ValueError: No objects to concatenate

In [409]:
df_drug_resuvatin_cancer.shape

(43185, 21)

In [84]:
df_drug_resuvatin_cancer.to_csv(project_path+'/data/processed_data/df_3.3.1_基线期_研究期_未换药.csv')

In [332]:
df_drug_resuvatin_cancer=pd.read_csv(project_path+'/data/processed_data/df_3.3.1_基线期_研究期_未换药.csv')
if 'Unnamed: 0' in df_drug_resuvatin_cancer.columns:
    del df_drug_resuvatin_cancer['Unnamed: 0']

### 日均剂量：5-20mg

In [333]:
df_drug_resuvatin_cancer['服药总剂量'] = df_drug_resuvatin_cancer['日剂量']*df_drug_resuvatin_cancer['用药时长']

In [334]:
for i in df_drug_resuvatin_cancer.患者编号.unique():
    temp = df_drug_resuvatin_cancer[df_drug_resuvatin_cancer['患者编号']==i]
    temp = temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    # 判断研究期日均剂量
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])
    temp = temp[(pd.to_datetime(temp['医嘱开始时间'])<=drug_start_time+datetime.timedelta(days=135)) &
                                (pd.to_datetime(temp['医嘱开始时间'])>=drug_start_time)]
    m_value = temp['服药总剂量'].sum()/temp['用药时长'].sum()
    if 5<=round(m_value,0)<=20:
        continue
    else:
        df_drug_resuvatin_cancer = df_drug_resuvatin_cancer[~ (df_drug_resuvatin_cancer['患者编号']==i)]

In [335]:
del df_drug_resuvatin_cancer['服药总剂量']
df_drug_resuvatin_cancer.shape

(48946, 21)

In [88]:
df_drug_resuvatin_cancer.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', '医嘱开始日期', '开药总剂量', '冠心病', '高脂血症', '冠心病_高脂血症',
       '首次用药时间'],
      dtype='object')

In [89]:
df_drug_resuvatin_cancer.data_source.value_counts()

人民医院     40164
世纪坛医院     8782
Name: data_source, dtype: int64

In [90]:
df_drug_resuvatin_cancer.患者编号.nunique()

9029

In [91]:
df_drug_resuvatin_cancer.to_csv(project_path+'/data/processed_data/df_3.3.2_日均剂量筛选.csv')

In [108]:
df_drug_resuvatin_cancer.用药时长.value_counts()

28.0     21383
14.0      6597
35.0      3818
18.0      3077
21.0      2535
         ...  
196.0        1
429.0        1
214.0        1
338.0        1
270.0        1
Name: 用药时长, Length: 340, dtype: int64

### 药物持有率>=50%

In [194]:
df_drug_resuvatin_keep = df_drug_resuvatin_cancer.copy().reset_index(drop=True)
temp_list=[]
for i in df_drug_resuvatin_cancer.患者编号.unique():
    temp = df_drug_resuvatin_cancer[df_drug_resuvatin_cancer['患者编号']==i]
    temp = temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    # 判断研究期用药天数
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])
    temp_r = temp[(pd.to_datetime(temp['医嘱开始时间'])<=drug_start_time+datetime.timedelta(days=135)) &
                                (pd.to_datetime(temp['医嘱开始时间'])>=drug_start_time)]
    k_value = temp_r['用药时长'].sum()/135
    if round(k_value,1)>=0.5:
        temp_list.append(temp)

In [195]:
df_drug_resuvatin_keep=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [196]:
df_drug_resuvatin_keep.shape

(35412, 21)

In [198]:
df_drug_resuvatin_keep.to_csv(project_path+'/data/processed_data/df_3.3.3_用药数据纳排.csv')

## 纳入血脂检查

### 提取血脂检查

In [440]:
'''
低密度胆固醇LDL-C
高密度胆固醇HDL-C
极低密度胆固醇VLDL-C
总胆固醇(TC)
甘油三酯(TG)
载脂蛋白A1 (ApoA1)
载脂蛋白B (ApoB)
脂蛋白(a) [Lp(a)]
'''

'\n低密度胆固醇LDL-C\n高密度胆固醇HDL-C\n极低密度胆固醇VLDL-C\n总胆固醇(TC)\n甘油三酯(TG)\n载脂蛋白A1 (ApoA1)\n载脂蛋白B (ApoB)\n脂蛋白(a) [Lp(a)]\n'

In [232]:
# 人民医院
df_check_1_BLE = df_check_1_T[['患者编号', '检验时间_date','HDL-C','LDL-C','CHO','TG','apo-A1','apo-B']]
# 修改列名
df_check_1_BLE = df_check_1_BLE.rename(columns={'CHO':'TC'})
# 删除行全空
df_check_1_BLE = df_check_1_BLE.reset_index(drop=True)
for i in range(df_check_1_BLE.shape[0]):
    if df_check_1_BLE.loc[i].notnull().sum()==2:
        df_check_1_BLE.drop([i],axis=0,inplace=True)

In [233]:
df_check_1_BLE.shape

(8769, 8)

In [234]:
df_check_1_BLE.head(1)

,患者编号,检验时间_date,HDL-C,LDL-C,TC,TG,apo-A1,apo-B
0,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16,1.1,2.42,3.87,2.89,119.5,86.4


In [235]:
df_check_2.columns

Index(['患者编号', '检验时间_date', '总胆固醇(TC)-静脉血', '天门冬氨酸氨基转移酶(AST)-静脉血',
       '丙氨酸氨基转移酶(ALT)-静脉血', '总胆红素(TBIL)-静脉血', '肌酐(Crea)-静脉血', '甘油三酯(TG)-静脉血',
       '尿素(Urea)-静脉血(空腹)', '尿素(Urea)-静脉血', '肌酸激酶(CK)-静脉血(空腹)', '肌酸激酶(CK)-静脉血',
       '葡萄糖(Glu)-末梢血', '葡萄糖(Glu)-静脉血(2小时)', '甘油三酯(TG)-静脉血(空腹)',
       '总胆固醇(TC)-静脉血(空腹)', '载脂蛋白B(apoB)-静脉血(空腹)', '载脂蛋白AI(apoAI)-静脉血(空腹)',
       '高密度脂蛋白胆固醇(HDL-C)-静脉血(空腹)', '低密度脂蛋白胆固醇(LDL-C)-静脉血(空腹)',
       '低密度脂蛋白胆固醇(LDL-C)-静脉血', '高密度脂蛋白胆固醇(HDL-C)-静脉血', '载脂蛋白AI(apoAI)-静脉血',
       '载脂蛋白B(apoB)-静脉血', '脂蛋白a(LPa)-静脉血', '微量总蛋白-24小时尿液', '总蛋白(TP)-尿液',
       '微量白蛋白(mAlb)-尿液'],
      dtype='object')

In [236]:
# 世纪坛医院
df_check_2_BLE = df_check_2[['患者编号', '检验时间_date','总胆固醇(TC)-静脉血','总胆固醇(TC)-静脉血(空腹)',
                             '低密度脂蛋白胆固醇(LDL-C)-静脉血', '低密度脂蛋白胆固醇(LDL-C)-静脉血(空腹)',
                             '高密度脂蛋白胆固醇(HDL-C)-静脉血','高密度脂蛋白胆固醇(HDL-C)-静脉血(空腹)',
                             '甘油三酯(TG)-静脉血','甘油三酯(TG)-静脉血(空腹)', 
                             '载脂蛋白AI(apoAI)-静脉血','载脂蛋白AI(apoAI)-静脉血(空腹)',
                            '载脂蛋白B(apoB)-静脉血','载脂蛋白B(apoB)-静脉血(空腹)',
                             '脂蛋白a(LPa)-静脉血']]
# 修改列名
df_check_2_BLE = df_check_2_BLE.rename(columns={'总胆固醇(TC)-静脉血':'TC','总胆固醇(TC)-静脉血(空腹)':'TC_empty',
                                    '低密度脂蛋白胆固醇(LDL-C)-静脉血':'LDL-C','低密度脂蛋白胆固醇(LDL-C)-静脉血(空腹)':'LDL-C_empty',
                                    '高密度脂蛋白胆固醇(HDL-C)-静脉血':'HDL-C','高密度脂蛋白胆固醇(HDL-C)-静脉血(空腹)':'HDL-C_empty',
                                    '甘油三酯(TG)-静脉血':'TG','甘油三酯(TG)-静脉血(空腹)':'TG_empty',
                                    '载脂蛋白AI(apoAI)-静脉血':'apo-A1','载脂蛋白AI(apoAI)-静脉血(空腹)':'apo-A1_empty',
                                    '载脂蛋白B(apoB)-静脉血':'apo-B','载脂蛋白B(apoB)-静脉血(空腹)':'apo-B_empty',
                                    '脂蛋白a(LPa)-静脉血':'LPa'})

In [237]:
# 合并空腹检查和静脉血检查
df_check_2_BLE['TC_empty']=df_check_2_BLE.apply(lambda x: x['TC'] if pd.isnull(x['TC_empty']) else x['TC_empty'],axis=1)
df_check_2_BLE['LDL-C_empty']=df_check_2_BLE.apply(lambda x: x['LDL-C'] if pd.isnull(x['LDL-C_empty']) else x['LDL-C_empty'],axis=1)
df_check_2_BLE['HDL-C_empty']=df_check_2_BLE.apply(lambda x: x['HDL-C'] if pd.isnull(x['HDL-C_empty']) else x['HDL-C_empty'],axis=1)
df_check_2_BLE['TG_empty']=df_check_2_BLE.apply(lambda x: x['TG'] if pd.isnull(x['TG_empty']) else x['TG_empty'],axis=1)
df_check_2_BLE['apo-A1_empty']=df_check_2_BLE.apply(lambda x: x['apo-A1'] if pd.isnull(x['apo-A1_empty']) else x['apo-A1_empty'],axis=1)
df_check_2_BLE['apo-B_empty']=df_check_2_BLE.apply(lambda x: x['apo-B'] if pd.isnull(x['apo-B_empty']) else x['apo-B_empty'],axis=1)

In [238]:
df_check_2_BLE=df_check_2_BLE[['患者编号', '检验时间_date','HDL-C_empty','LDL-C_empty','TC_empty','TG_empty','apo-A1_empty','apo-B_empty','LPa']]
df_check_2_BLE.columns=['患者编号', '检验时间_date','HDL-C','LDL-C','TC','TG','apo-A1','apo-B','LPa']

In [239]:
# 删除行全空
df_check_2_BLE = df_check_2_BLE.reset_index(drop=True)
for i in range(df_check_2_BLE.shape[0]):
    if df_check_2_BLE.loc[i].notnull().sum() == 2:
        df_check_2_BLE.drop([i],axis=0,inplace=True)

In [240]:
df_check_2_BLE.shape

(13208, 9)

In [242]:
df_check_2_BLE.head()

,患者编号,检验时间_date,HDL-C,LDL-C,TC,TG,apo-A1,apo-B,LPa
1,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-18,1.17,1.55,3.35,0.78,0.95,0.69,NaN
5,0012dfc2f29b979bf5c63fb4f11b7475,2019-11-22,1.04,3.23,5.03,1.67,NaN,NaN,NaN
8,00193379440c8cd81738464616c134e1,2018-11-01,1.18,2.29,3.83,0.70,1.03,0.75,NaN
10,00193379440c8cd81738464616c134e1,2019-05-18,1.13,1.37,3.07,0.77,0.96,0.57,NaN
11,0020640c4052f1dae97fcf39be390e1b,2019-04-01,1.54,1.69,3.79,1.30,NaN,NaN,NaN


In [243]:
# 合并血脂检查：人民 + 世纪坛
df_check_BLE = pd.concat([df_check_1_BLE,df_check_2_BLE],axis=0).reset_index(drop=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [244]:
df_check_BLE.shape

(21977, 9)

In [245]:
df_check_BLE.head(2)

,HDL-C,LDL-C,LPa,TC,TG,apo-A1,apo-B,患者编号,检验时间_date
0,1.10,2.42,NaN,3.87,2.89,119.5,86.4,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16
1,0.95,1.43,NaN,2.82,1.78,112.9,60.7,pat_2bb0dfeccae4278dbf399d6428a2573d,2019-04-10


In [340]:
df_check_BLE=df_check_BLE[['患者编号','检验时间_date','HDL-C','LDL-C','LPa','TC','TG','apo-A1','apo-B']]

In [341]:
df_check_BLE.to_csv(project_path+'/data/processed_data/df_3.4.1_提取血脂检查.csv')

### 计算非HDL-C

In [342]:
'''
非HDL-C（NonHDL-C）= 总胆固醇（TC）-高密度脂蛋白胆固醇（HDL-C）= LDL-C+VLDL-C
注：知道了TC、HDL-C、LDL-C或VLDL-C就可以计算得出非HDL-C。
'''

'\n非HDL-C（NonHDL-C）= 总胆固醇（TC）-高密度脂蛋白胆固醇（HDL-C）= LDL-C+VLDL-C\n注：知道了TC、HDL-C、LDL-C或VLDL-C就可以计算得出非HDL-C。\n'

In [343]:
 df_check_BLE['非HDL-C'] =  df_check_BLE.apply(lambda x: np.nan if (pd.isnull(x['TC']) or pd.isnull(x['HDL-C'])) else
                                                           x['TC']-x['HDL-C'], axis=1)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [344]:
df_check_BLE.head()

,患者编号,检验时间_date,HDL-C,LDL-C,LPa,TC,TG,apo-A1,apo-B,非HDL-C
0,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16,1.10,2.42,NaN,3.87,2.89,119.5,86.4,2.77
1,pat_2bb0dfeccae4278dbf399d6428a2573d,2019-04-10,0.95,1.43,NaN,2.82,1.78,112.9,60.7,1.87
2,pat_2bb0dfeccae4278dbf399d6428a2573d,2019-11-06,1.06,1.78,NaN,3.19,2.56,117.5,66.8,2.13
3,pat_043aba443d80d048105d8903d2034364,2018-06-05,1.30,3.28,NaN,4.78,1.48,NaN,NaN,3.48
4,pat_272f3f5730bfc5a361c461b4fde1dbb8,2018-08-15,1.24,2.22,NaN,4.25,1.15,125.7,79.2,3.01


In [345]:
df_check_BLE.to_csv(project_path+'/data/processed_data/df_3.4.2_计算非HDL-C.csv')

In [3]:
df_check_BLE=pd.read_csv(project_path+'/data/processed_data/疗效/df_3.4.2_计算非HDL-C.csv')
if 'Unnamed: 0' in df_check_BLE.columns:
    del df_check_BLE['Unnamed: 0']

### 血脂检查纳排

In [252]:
'''
血脂检查>=2
基线期和研究期：存在血脂数据
'''

'\n血脂检查>=2\n基线期和研究期：存在血脂数据\n'

In [390]:
# 读入处理半成品的瑞舒伐他汀数据
df_drug_resuvatin_BLE = df_drug_resuvatin_keep.copy().reset_index(drop=True)

In [391]:
# df_drug_resuvatin_BLE = df_drug_resuvatin_BLE.drop_duplicates(['患者编号','首次用药时间'],keep='first')[['data_source','患者编号','首次用药时间']]
# df_drug_resuvatin_BLE = pd.merge(df_drug_resuvatin_BLE,df_check_BLE,on='患者编号',how='left')

# pd.DataFrame(df_drug_resuvatin_BLE.患者编号.value_counts()).to_csv(project_path+'/data/processed_data/df_temp_血脂纳排前统计.csv')

# df_drug_resuvatin_BLE.to_csv(project_path+'/data/processed_data/df_3.4.3_血脂纳排前用药数据.csv')

In [392]:
df_drug_resuvatin_BLE.shape

(35412, 21)

In [393]:
# 删除全列为空
def del_allNull(df_model):
    # 血脂检查项
    ble_list = ['HDL-C','LDL-C','TC','TG','apo-A1','apo-B']
    df_ble = df_model[['患者编号','检验时间_date']+ble_list]
    # 列全空
    for i in df_ble.columns:
        if df_ble[i].isnull().sum()== df_ble.shape[0]:
            del df_ble[i]
    # 行全空
    df_ble = df_ble.reset_index(drop=True)
    for i in range(df_ble.shape[0]):
        if df_ble.loc[i].isnull().sum()== (df_ble.shape[1]-2):
            df_ble.drop([i],axis=0,inplace=True)
    return df_ble.reset_index(drop=True)

In [394]:
temp_list=[]
for i in df_drug_resuvatin_BLE.患者编号.unique():
    # 患者用药
    temp = df_drug_resuvatin_BLE[df_drug_resuvatin_BLE['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])   # 初始开药时间
    # 患者血脂检查
    temp_check = df_check_BLE[df_check_BLE['患者编号']==i]
    temp_check = temp_check.sort_values(by=['检验时间_date'],ascending=True).reset_index(drop=True)
    temp_check_nt = del_allNull(temp_check) # 不全为空的血脂检查
    if temp_check_nt.shape[0]<2:
        continue
    # 基线期-血脂检查
    temp_check_b = temp_check_nt[(pd.to_datetime(temp_check_nt['检验时间_date'])>=drug_start_time-datetime.timedelta(days=90)) &
                                (pd.to_datetime(temp_check_nt['检验时间_date'])<=drug_start_time)]
    # 研究期-血脂检查
    temp_check_r = temp_check_nt[(pd.to_datetime(temp_check_nt['检验时间_date'])<=drug_start_time+datetime.timedelta(days=135)) &
                                (pd.to_datetime(temp_check_nt['检验时间_date'])>drug_start_time+datetime.timedelta(days=45))]
    if (temp_check_b.shape[0]>=1) & (temp_check_r.shape[0]>=1):
        temp_list.append(temp)

In [395]:
df_drug_resuvatin_BLE=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [396]:
df_drug_resuvatin_BLE.患者编号.nunique()

164

In [397]:
df_drug_resuvatin_BLE.drop_duplicates(['患者编号','data_source'],keep='first').data_source.value_counts()

人民医院     141
世纪坛医院     23
Name: data_source, dtype: int64

In [398]:
df_drug_resuvatin_BLE.shape

(1619, 21)

In [399]:
df_drug_resuvatin_BLE.data_source.value_counts()

人民医院     1424
世纪坛医院     195
Name: data_source, dtype: int64

In [207]:
df_drug_resuvatin_BLE.to_csv(project_path+'/data/processed_data/df_3.4.3_血脂检查纳排.csv')

In [499]:
df_drug_resuvatin_BLE_del=df_drug_resuvatin_keep[~ df_drug_resuvatin_keep.患者编号.isin(df_drug_resuvatin_BLE.患者编号.unique())]

In [500]:
df_drug_resuvatin_BLE_del.shape

(33935, 20)

In [501]:
df_drug_resuvatin_BLE_del.患者编号.nunique()

3456

In [200]:
df_drug_resuvatin_BLE_del.to_csv(project_path+'/data/processed_data/df_3.4.3_血脂检查删除数据.csv')

NameError: name 'df_drug_resuvatin_BLE_del' is not defined

## 人口学特征

In [402]:
'''
AGE=患者初次处方京新和/或可定日期-患者的出生日期
'''

'\nAGE=患者初次处方京新和/或可定日期-患者的出生日期\n'

### 人民医院--人口学数据

In [403]:
df_popu_1 = pd.read_excel(r'./data/raw_data/人民医院/1-患者基本信息.xlsx')

In [404]:
df_popu_1.shape

(44998, 3)

In [405]:
df_popu_1.columns

Index(['患者编号', '性别', '出生日期'], dtype='object')

### 世纪坛医院--人口学数据

In [406]:
# 世纪坛医院--人口学数据
df_popu_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                              sheet_name='患者')

In [407]:
df_popu_2.shape

(61142, 11)

In [408]:
df_popu_2.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'patient_SN', '性别', '出生日期', '年龄', '民族',
       '既往是否有疾病史', '历史总门诊就诊次数', '历史总急诊就诊次数', '历史总住院就诊次数'],
      dtype='object')

In [409]:
df_popu_2 = df_popu_2[['patient_SN', '性别', '出生日期']]
df_popu_2.rename(columns={'patient_SN':'患者编号'},inplace=True)

### 合并人口学特征

In [410]:
df_popu = pd.concat([df_popu_1,df_popu_2],axis=0).reset_index(drop=True)

In [411]:
df_popu.shape

(106140, 3)

In [412]:
# 删除重复数据
df_popu = df_popu.drop_duplicates(df_popu.columns, keep='first')

In [413]:
df_popu.shape

(65804, 3)

In [414]:
df_popu.head()

,患者编号,性别,出生日期
0,pat_0d9297a346707bce01f0b4637484efe9,女性,1956
1,pat_0ebd83a8c3990a103016a650f84616f8,女性,1962
2,pat_13da443c402ad02c9e9de0c5c0751eda,女性,1973
3,pat_2c40bef1b35d17984b1d393c6be55688,女性,1952
4,pat_0f6bf632666460422c4923f591ede61f,女性,1943


In [415]:
df_popu.to_csv(project_path+'/data/processed_data/df_3.5.3_合并人口学特征.csv')

### 并入人口学特征

In [416]:
# 性别：男-1；女-0
df_popu['性别'] = df_popu.性别.apply(lambda x: 1 if re.match('男',x) else 0)

In [417]:
df_popu.性别.value_counts()

1    34957
0    30847
Name: 性别, dtype: int64

In [418]:
# 提取出生年
df_popu['出生日期'] = df_popu.出生日期.apply(lambda x: np.nan if pd.isnull(x) else
                                     str(x).split('-')[0] if '-' in str(x) else x)

In [410]:
# 计算年龄
df_drug_resuvatin_popu = df_drug_resuvatin_BLE.copy().reset_index(drop=True)
df_drug_start_time = df_drug_resuvatin_popu.sort_values(['患者编号','医嘱开始时间']).drop_duplicates(['患者编号'],keep='first')
df_drug_start_time = df_drug_start_time[['患者编号','医嘱开始时间']].rename(columns={'医嘱开始时间':'初次用药时间'})
df_drug_start_time['初次用药年'] =df_drug_start_time.初次用药时间.apply(lambda x: str(x).split('-')[0])
# 并入到用药数据
df_drug_resuvatin_popu = pd.merge(df_drug_resuvatin_popu,df_drug_start_time,on='患者编号',how='left')
# 并入人口学特征
df_drug_resuvatin_popu = pd.merge(df_drug_resuvatin_popu,df_popu,on='患者编号',how='left')
# 计算年龄
df_drug_resuvatin_popu['年龄'] = df_drug_resuvatin_popu.apply(lambda x:  np.nan if pd.isnull(x['出生日期']) else
                                                                        int(x['初次用药年'])-int(x['出生日期']) ,axis=1)

NameError: name 'df_popu' is not defined

In [528]:
df_drug_resuvatin_popu.shape

(1420, 25)

In [529]:
df_drug_resuvatin_popu.to_csv(project_path+'/data/processed_data/df_3.5.4_并入人口学特征.csv')

PermissionError: [Errno 13] Permission denied: 'D:\\PycharmProjects\\Rosu_Beijing_renmin_hospital/data/processed_data/df_3.5.4_并入人口学特征.csv'

## 个人史既往史

In [530]:
'''
初次处方京新和/或可定日期(+1个月)前有过高血压、糖尿病、肝胆疾病等相关诊断，明确患者的疾病史
'''

'\n初次处方京新和/或可定日期(+1个月)前有过高血压、糖尿病、肝胆疾病等相关诊断，明确患者的疾病史\n'

### 人民医院--既往史

In [531]:
df_history_1 = pd.read_excel(r'./data/raw_data/人民医院/5-病史（1诉5史）.xlsx')

In [532]:
# 删除重复值
df_history_1 = df_history_1.drop_duplicates(['患者编号','主诉'],keep='first')[['患者编号','主诉']]
# 删除空值
df_history_1 = df_history_1[df_history_1.主诉.notnull()]

In [533]:
df_history_1.shape

(83658, 2)

In [534]:
df_history_1.head(1)

,患者编号,主诉
0,pat_0d9297a346707bce01f0b4637484efe9,高血压平时在积水潭医院诊疗


In [535]:
temp_list=[]
for i in df_history_1.患者编号.unique():
    temp=df_history_1[df_history_1['患者编号']==i]
    record_list=temp['主诉'].to_list()
    record=';'.join(record_list)
    record_list=re.split(',|，|;|。',record)
    for j in record_list:
        # 吸烟
        if re.match('[^无未否戒]*烟',j):
            temp['吸烟史']=1
        # 饮酒
        if re.match('[^无未否戒]*酒[^精]',j):
            temp['饮酒史']=1
    temp_list.append(temp)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [536]:
df_history_1=pd.concat(temp_list,axis=0)
del temp_list
df_history_1=df_history_1.drop_duplicates(['患者编号'],keep='first')[['患者编号','吸烟史','饮酒史']]
df_history_1=df_history_1.fillna(0) # 缺失值填充0

In [537]:
df_history_1.吸烟史.value_counts()

0.0    36601
1.0       29
Name: 吸烟史, dtype: int64

In [538]:
df_history_1.to_csv(project_path+'/data/processed_data/df_3.6.1_人民医院_既往史.csv')

### 提取高血压糖尿病

In [539]:
df_diagnose.columns

Index(['患者编号', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称'], dtype='object')

In [540]:
# 提取高血压
df_diagnose_EH = df_diagnose[df_diagnose.诊断名称.str.contains('高血压')].drop_duplicates(['患者编号'],keep='first')
# 提取糖尿病
df_diagnose_DM = df_diagnose[df_diagnose.诊断名称.str.contains('糖尿病')].drop_duplicates(['患者编号'],keep='first')

In [541]:
print(df_diagnose_EH.shape)
print(df_diagnose_DM.shape)

(39706, 5)
(14823, 5)


In [542]:
df_diagnose_EH.head(1)

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
48,pat_12b28ceaa0f528b29568b2ce795c9e75,高血压(或心绞痛)伴动脉粥样硬化并高脂血症,2017-12-04 15:26:49,否,门诊诊断


### 并入个人史既往史

In [543]:
# 并入个人史
df_drug_resuvatin_popu = pd.merge(df_drug_resuvatin_popu,df_history_1,on='患者编号',how='left')
df_drug_resuvatin_popu['吸烟史']=df_drug_resuvatin_popu['吸烟史'].apply(lambda x: 0 if pd.isnull(x) else x)
df_drug_resuvatin_popu['饮酒史']=df_drug_resuvatin_popu['饮酒史'].apply(lambda x: 0 if pd.isnull(x) else x)

In [544]:
# 并入高血压、糖尿病
df_drug_resuvatin_popu['高血压'] = df_drug_resuvatin_popu['患者编号'].apply(lambda x: 1 if x in df_diagnose_EH.患者编号.unique() else 0)
df_drug_resuvatin_popu['糖尿病'] = df_drug_resuvatin_popu['患者编号'].apply(lambda x: 1 if x in df_diagnose_DM.患者编号.unique() else 0)

In [40]:
df_drug_resuvatin_popu.shape

(1420, 29)

In [545]:
df_drug_resuvatin_popu.to_csv(project_path+'/data/processed_data/df_3.6.3_并入个人史既往史.csv')

## ASCVD预防分级

In [77]:
'''
中低危、高危和极高危
由于缺乏患者的BMI、血压值和吸烟情况，对于缺失信息的患者的等级划分采用就低原则（信息缺失视为未发生）
'''

'\n中低危、高危和极高危\n由于缺乏患者的BMI、血压值和吸烟情况，对于缺失信息的患者的等级划分采用就低原则（信息缺失视为未发生）\n'

In [251]:
df_drug_resuvatin_popu=pd.read_csv(project_path+'/data/processed_data/df_3.6.3_并入个人史既往史.csv')
if 'Unnamed: 0' in df_drug_resuvatin_popu.columns:
    del df_drug_resuvatin_popu['Unnamed: 0']
df_drug_resuvatin_ASCVD = df_drug_resuvatin_popu.copy().reset_index(drop=True)

In [252]:
df_drug_resuvatin_ASCVD.shape

(1420, 29)

In [253]:
df_drug_resuvatin_ASCVD.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', 'drug_start_date', '开药总剂量', '冠心病', '高脂血症',
       '冠心病_高脂血症', '初次用药时间', '初次用药年', '性别', '出生日期', '年龄', '吸烟史', '饮酒史', '高血压',
       '糖尿病'],
      dtype='object')

### 第一步分级

In [254]:
'''
已经诊断为动脉粥样硬化性心血管疾病（atherosclerotic cardiovascular disease, ASCVD）者直接列为极高危人群； 
符合如下条件之一者直接列为高危人群：
（1）基线LDL-C ≥ 4.9 mmol/L 或 TC ≥ 7.2 mmol/L。
（2）糖尿病患者 1.8 mmol/L ≤ LDL-C<4.9 mmol/L（或）3.1 mmol/L ≤ TC<7.2 mmol/L 且年龄≥ 40 岁
'''

'\n已经诊断为动脉粥样硬化性心血管疾病（atherosclerotic cardiovascular disease, ASCVD）者直接列为极高危人群； \n符合如下条件之一者直接列为高危人群：\n（1）基线LDL-C ≥ 4.9 mmol/L 或 TC ≥ 7.2 mmol/L。\n（2）糖尿病患者 1.8 mmol/L ≤ LDL-C<4.9 mmol/L（或）3.1 mmol/L ≤ TC<7.2 mmol/L 且年龄≥ 40 岁\n'

#### ASCVD诊断

In [255]:
'''
ASCVD诊断：
急性冠状动脉综合征：急性冠脉综合征
稳定型心绞痛：稳定性心绞痛。~不稳定型心绞痛
冠状动脉血运重建
冠心病(冠状动脉粥样硬化性心脏病)：隐匿型冠心病(无症状心肌缺血)、心绞痛型冠心病、心肌梗死型(心肌埂塞)、缺血性心肌病型(缺血性心力衰竭；缺血性心脏病)、猝死型心脏病
动脉粥样硬化性脑卒中
短暂性脑缺血发作：短暂性脑缺血
外周动脉血管疾病：外周动脉硬化、周围血管疾病
'''

'\nASCVD诊断：\n急性冠状动脉综合征：急性冠脉综合征\n稳定型心绞痛：稳定性心绞痛。~不稳定型心绞痛\n冠状动脉血运重建\n冠心病(冠状动脉粥样硬化性心脏病)：隐匿型冠心病(无症状心肌缺血)、心绞痛型冠心病、心肌梗死型(心肌埂塞)、缺血性心肌病型(缺血性心力衰竭；缺血性心脏病)、猝死型心脏病\n动脉粥样硬化性脑卒中\n短暂性脑缺血发作：短暂性脑缺血\n外周动脉血管疾病：外周动脉硬化、周围血管疾病\n'

In [256]:
df_diagnose.head()

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
0,pat_0d9297a346707bce01f0b4637484efe9,高胆固醇血症,2019-12-31 07:54:05,否,门诊诊断
1,pat_0ebd83a8c3990a103016a650f84616f8,高胆固醇血症,2018-03-11 11:06:21,否,门诊诊断
2,pat_13da443c402ad02c9e9de0c5c0751eda,动脉粥样硬化并高脂血症,2018-01-08 16:29:49,否,门诊诊断
3,pat_13da443c402ad02c9e9de0c5c0751eda,动脉粥样硬化并高脂血症,2018-03-22 15:00:58,否,门诊诊断
4,pat_13da443c402ad02c9e9de0c5c0751eda,动脉粥样硬化并高脂血症,2018-06-26 14:08:41,否,门诊诊断


In [257]:
df_diagnose_ASCVD_1 = df_diagnose[df_diagnose.诊断名称.str.contains('急性冠脉综合征')]

In [258]:
df_diagnose_ASCVD_2 = df_diagnose[df_diagnose.诊断名称.str.contains('稳定型心绞痛|稳定性心绞痛')]

In [259]:
df_diagnose_ASCVD_3 = df_diagnose[df_diagnose.诊断名称.str.contains('冠状动脉血运重建')]

In [260]:
df_diagnose_ASCVD_4 = df_diagnose[df_diagnose.诊断名称.str.contains('隐匿型|心绞痛|心肌梗死|心肌埂塞|缺血性心肌病|缺血性心力衰竭|缺血性心脏病|猝死') &
                                 df_diagnose.诊断名称.str.contains('冠心病|冠状动脉粥样硬化性心脏病')]

In [261]:
df_diagnose_ASCVD_5 = df_diagnose[df_diagnose.诊断名称.str.contains('动脉粥样硬化性脑卒中')]

In [262]:
df_diagnose_ASCVD_6 = df_diagnose[df_diagnose.诊断名称.str.contains('短暂性脑缺血|外周动脉血管疾病|外周动脉硬化|周围血管疾病')]

In [263]:
# 合并ASCVD诊断
df_diagnose_ASCVD = pd.concat([df_diagnose_ASCVD_1,df_diagnose_ASCVD_2],axis=0)
df_diagnose_ASCVD = pd.concat([df_diagnose_ASCVD,df_diagnose_ASCVD_3],axis=0)
df_diagnose_ASCVD = pd.concat([df_diagnose_ASCVD,df_diagnose_ASCVD_4],axis=0)
df_diagnose_ASCVD = pd.concat([df_diagnose_ASCVD,df_diagnose_ASCVD_5],axis=0)
df_diagnose_ASCVD = pd.concat([df_diagnose_ASCVD,df_diagnose_ASCVD_6],axis=0)
# 删除重复患者
df_diagnose_ASCVD = df_diagnose_ASCVD.drop_duplicates(['患者编号'],keep='first')[['患者编号']]
df_diagnose_ASCVD['ASCVD'] = '极高危'

In [264]:
df_drug_resuvatin_ASCVD=pd.merge(df_drug_resuvatin_ASCVD,df_diagnose_ASCVD,on='患者编号',how='left')

#### 高危人群诊断

In [265]:
df_check_BLE=pd.read_csv(project_path+'/data/processed_data/df_3.4.2_计算非HDL-C.csv')
if 'Unnamed: 0' in df_check_BLE.columns:
    del df_check_BLE['Unnamed: 0']

In [266]:
temp_list=[]
for i in df_drug_resuvatin_ASCVD.患者编号.unique():
    # 患者用药
    temp = df_drug_resuvatin_ASCVD[df_drug_resuvatin_ASCVD['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])   # 初始开药时间
    age = temp.loc[0,'年龄'] 
    DM_history = temp.loc[0,'糖尿病']
    # 判断是否有ASCVD诊断
    if pd.notnull(temp.loc[0,'ASCVD']):
        temp_list.append(temp)
        continue
    # 患者血脂检查
    temp_check = df_check_BLE[df_check_BLE['患者编号']==i]
    temp_check = temp_check.sort_values(by=['检验时间_date'],ascending=True).reset_index(drop=True)
    temp_check_nt = del_allNull(temp_check) # 不全为空的血脂检查
    if temp_check_nt.shape[0]>=1:
        # 基线期-血脂检查
        temp_check_b = temp_check_nt[(pd.to_datetime(temp_check_nt['检验时间_date'])>=drug_start_time-datetime.timedelta(days=90)) &
                                    (pd.to_datetime(temp_check_nt['检验时间_date'])<=drug_start_time)]
        if temp_check_b.shape[0]>=1:
            temp_check_b = temp_check_b.sort_values(['检验时间_date'],ascending=True).reset_index(drop=True)
            for j in range(temp_check_b.shape[0]):
                if (temp_check_b.loc[j,'LDL-C'] >=4.9) or (temp_check_b.loc[j,'TC']>=7.2):
                    temp['ASCVD']='高危'
                    break
                elif (1.8<=temp_check_b.loc[j,'LDL-C'] <4.9) or (3.1<=temp_check_b.loc[j,'TC'] <7.2):
                    if (age>=40) & (DM_history==1):
                        temp['ASCVD']='高危'
                        break
    temp_list.append(temp)

In [267]:
df_drug_resuvatin_ASCVD = pd.concat(temp_list,axis=0)
del temp_list

In [268]:
df_drug_resuvatin_ASCVD.ASCVD.value_counts()

高危     421
极高危     96
Name: ASCVD, dtype: int64

In [269]:
df_drug_resuvatin_ASCVD.shape

(1420, 30)

In [271]:
df_drug_resuvatin_ASCVD.to_csv(project_path+'/data/processed_data/df_3.7.1_第一步_ASCVD分级.csv')

### 第二步分级

#### 统计危险因素个数

In [280]:
df_drug_resuvatin_ASCVD = df_drug_resuvatin_ASCVD.reset_index(drop=True)
temp_list=[]
for i in df_drug_resuvatin_ASCVD.患者编号.unique():
    count=0
    # 患者用药
    temp = df_drug_resuvatin_ASCVD[df_drug_resuvatin_ASCVD['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])   # 初始开药时间
    gender = temp.loc[0,'性别'] 
    age = temp.loc[0,'年龄'] 
    DM_history = temp.loc[0,'糖尿病']
    # 基线期-血脂检查
    temp_check = df_check_BLE[df_check_BLE['患者编号']==i]
    temp_check = temp_check.sort_values(by=['检验时间_date'],ascending=True).reset_index(drop=True)
    if temp_check.shape[0]>=1:
        temp_check_b = temp_check[(pd.to_datetime(temp_check['检验时间_date'])>=drug_start_time-datetime.timedelta(days=90)) &
                                    (pd.to_datetime(temp_check['检验时间_date'])<=drug_start_time)]
        if temp_check_b.shape[0]>=1:
            temp_check_b = temp_check_b.sort_values(['检验时间_date'],ascending=False).reset_index(drop=True)
            if temp_check_b.loc[0,'HDL-C']<1:
                count+=1
    # 吸烟
    if temp['吸烟史'].sum()>0:
        count+=1
    # 性别
    if (gender==1) & (age>=45):
        count+=1
    elif (gender==0) & (age>=55):
        count +=1
    print(count)
    temp['危险因素个数']=count
    temp_list.append(temp)

1
0
0
1
1
1
1
1
0
1
1
1
1
2
1
1
1
1
1
1
1
1
1
0
1
1
2
1
1
1
2
2
2
2
1
2
1
1
1
1
1
0
2
1
1
1
1
2
1
2
0
1
1
0
1
1
0
1
1
1
2
1
1
1
2
1
1
1
1
0
0
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
2
1
0
1
1
1
1
1
1
2
2
1
1
1
1
1
1
2
1
1
2
0
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
2
1
1
1
1
2
1
2
1
2
1
2
1
1
1
1


In [282]:
df_drug_resuvatin_ASCVD = pd.concat(temp_list,axis=0)
del temp_list

In [283]:
df_drug_resuvatin_ASCVD.危险因素个数.value_counts()

1    1053
2     256
0     111
Name: 危险因素个数, dtype: int64

#### 第二步：血脂-ASCVD分级

In [284]:
df_drug_resuvatin_ASCVD = df_drug_resuvatin_ASCVD.reset_index(drop=True)
temp_list=[]
for i in df_drug_resuvatin_ASCVD.患者编号.unique():
    count=0
    # 患者用药
    temp = df_drug_resuvatin_ASCVD[df_drug_resuvatin_ASCVD['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])   # 初始开药时间
    gender = temp.loc[0,'性别'] 
    age = temp.loc[0,'年龄'] 
    EH_history = temp.loc[0,'高血压']
    if pd.notnull(temp.loc[0,'ASCVD']):
        temp_list.append(temp)
        continue
    # 基线期-血脂检查
    temp_check = df_check_BLE[df_check_BLE['患者编号']==i]
    temp_check = temp_check.sort_values(by=['检验时间_date'],ascending=True).reset_index(drop=True)
    if temp_check.shape[0]>=1:
        temp_check_b = temp_check[(pd.to_datetime(temp_check['检验时间_date'])>=drug_start_time-datetime.timedelta(days=90)) &
                                    (pd.to_datetime(temp_check['检验时间_date'])<=drug_start_time)]
        temp_check_b = temp_check_b.sort_values(by=['检验时间_date'],ascending=0).reset_index(drop=True)
        if temp_check_b.shape[0]>=1:
            # 第3列ASCVD分级
            if (3.4<=temp_check_b.loc[0,'LDL-C']<4.9) or (5.2<=temp_check_b.loc[0,'TC']<7.2):
                # 无高血压
                if EH_history==0:
                    if temp['危险因素个数'].max()>=2:
                        temp['ASCVD']='中危'
                    elif temp['危险因素个数'].max()<=1:
                        temp['ASCVD']='低危'
                # 有高血压
                if EH_history==1:
                    if temp['危险因素个数'].max()>=2:
                        temp['ASCVD']='高危'
                    elif temp['危险因素个数'].max()==1:
                        temp['ASCVD']='中危'
                    elif temp['危险因素个数'].max()==0:
                        temp['ASCVD']='低危'
            
            # 第2列ASCVD分级
            if (2.6<=temp_check_b.loc[0,'LDL-C']<3.4) or (4.1<=temp_check_b.loc[0,'TC']<5.2):
                # 无高血压
                if EH_history==0:
                    if temp['危险因素个数'].max()>2:
                        temp['ASCVD']='中危'
                    elif temp['危险因素个数'].max()<=2:
                        temp['ASCVD']='低危'
                # 有高血压
                if EH_history==1:
                    if temp['危险因素个数'].max()>=2:
                        temp['ASCVD']='高危'
                    elif temp['危险因素个数'].max()==1:
                        temp['ASCVD']='中危'
                    elif temp['危险因素个数'].max()==0:
                        temp['ASCVD']='低危'
            
            # 第1列ASCVD分级
            if (1.8<=temp_check_b.loc[0,'LDL-C']<2.6) or (3.1<=temp_check_b.loc[0,'TC']<4.1):
                # 无高血压
                if EH_history==0:
                    temp['ASCVD']='低危'
                # 有高血压
                if EH_history==1:
                    if temp['危险因素个数'].max()>2:
                        temp['ASCVD']='高危'
                    elif temp['危险因素个数'].max()==2:
                        temp['ASCVD']='中危'
                    elif temp['危险因素个数'].max()<=1:
                        temp['ASCVD']='低危'
    temp_list.append(temp)

In [285]:
df_drug_resuvatin_ASCVD = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [286]:
df_drug_resuvatin_ASCVD.ASCVD.value_counts()

高危     444
中危     428
低危     393
极高危     96
Name: ASCVD, dtype: int64

In [287]:
df_drug_resuvatin_ASCVD.shape

(1420, 31)

In [289]:
df_drug_resuvatin_ASCVD.to_csv(project_path+'/data/processed_data/df_3.7.2_第二步_ASCVD分级.csv')

### 第三步分级

In [305]:
# 余生危险
df_drug_resuvatin_ASCVD['余生危险'] = df_drug_resuvatin_ASCVD.apply(lambda x: np.nan if pd.isnull(x['ASCVD']) else
                                                                    1 if (x['年龄']<55) & (x['ASCVD']=='中危') else np.nan ,axis=1)

In [306]:
df_drug_resuvatin_ASCVD['余生危险'].value_counts()

1.0    41
Name: 余生危险, dtype: int64

In [316]:
# 任意两项以上为高危
df_drug_resuvatin_ASCVD = df_drug_resuvatin_ASCVD.reset_index(drop=True)
temp_list=[]
for i in df_drug_resuvatin_ASCVD.患者编号.unique():
    count=0
    # 患者用药
    temp = df_drug_resuvatin_ASCVD[df_drug_resuvatin_ASCVD['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    drug_start_time = pd.to_datetime(temp.loc[0,'医嘱开始时间'])   # 初始开药时间
    gender = temp.loc[0,'性别'] 
    age = temp.loc[0,'年龄'] 
    DM_history = temp.loc[0,'糖尿病']
    if (temp.loc[0,'ASCVD']=='极高危') or (temp.loc[0,'ASCVD']=='高危'):
        temp_list.append(temp)
        continue
    # 基线期-血脂检查
    temp_check = df_check_BLE[df_check_BLE['患者编号']==i]
    temp_check = temp_check.sort_values(by=['检验时间_date'],ascending=True).reset_index(drop=True)
    if temp_check.shape[0]>=1:
        temp_check_b = temp_check[(pd.to_datetime(temp_check['检验时间_date'])>=drug_start_time-datetime.timedelta(days=90)) &
                                    (pd.to_datetime(temp_check['检验时间_date'])<=drug_start_time)]
        if temp_check_b.shape[0]>=1:
            temp_check_b = temp_check_b.sort_values(['检验时间_date'],ascending=False).reset_index(drop=True)
            if temp_check_b.loc[0,'非HDL-C']>=5.2:
                count+=1
            if temp_check_b.loc[0,'HDL-C']<1:
                count+=1
    # 吸烟
    if temp['吸烟史'].sum()>0:
        count+=1
    if count >=2:
        print(count)
        temp['ASCVD']='高危'
    temp_list.append(temp)

In [317]:
df_drug_resuvatin_ASCVD = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [318]:
df_drug_resuvatin_ASCVD.shape

(1420, 32)

In [320]:
df_drug_resuvatin_ASCVD.to_csv(project_path+'/data/processed_data/df_3.7.3_第三步_ASCVD分级.csv')

In [10]:
df_drug_resuvatin_ASCVD=pd.read_csv(project_path+'/data/processed_data/疗效/df_3.7.3_第三步_ASCVD分级.csv')
if 'Unnamed: 0' in df_drug_resuvatin_ASCVD.columns:
    del df_drug_resuvatin_ASCVD['Unnamed: 0']

In [11]:
df_drug_resuvatin_ASCVD.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', 'drug_start_date', '开药总剂量', '冠心病', '高脂血症',
       '冠心病_高脂血症', '初次用药时间', '初次用药年', '性别', '出生日期', '年龄', '吸烟史', '饮酒史', '高血压',
       '糖尿病', 'ASCVD', '危险因素个数', '余生危险'],
      dtype='object')

## 疗效评价

In [12]:
'''
极高危人群：
    LDL-C目标值＜1.8mmol/L(70mg/dl); 非HDL-C<2.6
    目标为LDL-C降幅至少50%
高危人群：
    LDL-C目标值＜2.6mmol/L(100mg/dl); 非HDL-C<3.4
    目标为LDL-C降幅至少50%
低危、中危人群：
    LDL-C目标值＜3.4/L(100mg/dl); 非HDL-C<4.1
    目标为LDL-C降幅至少50%
'''

'\n极高危人群：\n    LDL-C目标值＜1.8mmol/L(70mg/dl); 非HDL-C<2.6\n    目标为LDL-C降幅至少50%\n高危人群：\n    LDL-C目标值＜2.6mmol/L(100mg/dl); 非HDL-C<3.4\n    目标为LDL-C降幅至少50%\n低危、中危人群：\n    LDL-C目标值＜3.4/L(100mg/dl); 非HDL-C<4.1\n    目标为LDL-C降幅至少50%\n'

In [58]:
df_drug_resuvatin_efficacy = df_drug_resuvatin_ASCVD.copy().reset_index(drop=True)

In [59]:
def judge_efficacy(ascvd,r_ldl,r_非hdl,b_ldl,b_非hdl):
    # 是否达到目标值
    if ascvd == '极高危':
        if (r_ldl<1.8) or (r_非hdl<2.6):
            return True
    elif ascvd == '高危':
        if (r_ldl<2.6) or (r_非hdl<3.4):
            return True
    elif (ascvd == '中危') or (ascvd=='低危'):
        if (r_ldl<3.4) or (r_非hdl<4.1):
            return True
    # 是否下降50%
    if abs(r_ldl-b_ldl)/b_ldl >=0.5:
        return True
    elif abs(r_非hdl-b_非hdl)/b_非hdl >=0.5:
        return True

In [60]:
df_drug_resuvatin_efficacy=df_drug_resuvatin_efficacy.sort_values(by=['医嘱开始时间'],ascending=True)
df_drug_resuvatin_efficacy=df_drug_resuvatin_efficacy.drop_duplicates(['患者编号'],keep='first').reset_index(drop=True)

In [61]:
df_drug_resuvatin_efficacy.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', 'drug_start_date', '开药总剂量', '冠心病', '高脂血症',
       '冠心病_高脂血症', '初次用药时间', '初次用药年', '性别', '出生日期', '年龄', '吸烟史', '饮酒史', '高血压',
       '糖尿病', 'ASCVD', '危险因素个数', '余生危险'],
      dtype='object')

In [62]:
temp_check

,患者编号,检验时间_date,HDL-C,LDL-C,LPa,TC,TG,apo-A1,apo-B,非HDL-C
0,pat_0a31a5b06ad2a27c6d4dd950266235d9,2019-07-18,1.18,2.76,NaN,4.44,1.64,NaN,NaN,3.26
1,pat_0a31a5b06ad2a27c6d4dd950266235d9,2019-12-28,1.33,2.71,NaN,4.36,1.48,NaN,NaN,3.03
2,pat_0a31a5b06ad2a27c6d4dd950266235d9,2020-04-21,1.35,2.36,NaN,4.07,1.75,NaN,NaN,2.72


In [63]:
for i in range(df_drug_resuvatin_efficacy.shape[0]):
    patient_id = df_drug_resuvatin_efficacy.loc[i,'患者编号']
    # 患者用药
    drug_start_time = pd.to_datetime(df_drug_resuvatin_efficacy.loc[i,'医嘱开始时间'])   # 初始开药时间
    ascvd = df_drug_resuvatin_efficacy.loc[i,'ASCVD']
    
    # 血脂检查
    temp_check = df_check_BLE[df_check_BLE['患者编号']==patient_id]
    temp_check = temp_check.sort_values(by=['检验时间_date'],ascending=True).reset_index(drop=True)
    
    if temp_check.shape[0]>=1:
        # 基线期-血脂检查
        temp_check_b = temp_check[(pd.to_datetime(temp_check['检验时间_date'])>=drug_start_time-datetime.timedelta(days=90)) &
                                    (pd.to_datetime(temp_check['检验时间_date'])<=drug_start_time)]
        if temp_check_b.shape[0]>=1:
            temp_check_b = temp_check_b.sort_values(['检验时间_date'],ascending=False).reset_index(drop=True)
            b_ldl = temp_check_b.loc[0,'LDL-C']
            b_非hdl = temp_check_b.loc[0,'非HDL-C']
            df_drug_resuvatin_efficacy.loc[i,'基线期_LDL-C']=b_ldl
            df_drug_resuvatin_efficacy.loc[i,'基线期_非HDL-C']=b_非hdl
        else:
            b_ldl = np.nan
            b_非hdl = np.nan
            df_drug_resuvatin_efficacy.loc[i,'基线期_LDL-C']=b_ldl
            df_drug_resuvatin_efficacy.loc[i,'基线期_非HDL-C']=b_非hdl
        # 研究期-血脂检查
        temp_check_r = temp_check[(pd.to_datetime(temp_check['检验时间_date'])>=drug_start_time+datetime.timedelta(days=45)) &
                                    (pd.to_datetime(temp_check['检验时间_date'])<=drug_start_time+datetime.timedelta(days=135))]
        if temp_check_r.shape[0]>=1:
            temp_check_r = temp_check_r.sort_values(['检验时间_date'],ascending=True).reset_index(drop=True)
            for j in range(temp_check_r.shape[0]):
                r_ldl = temp_check_r.loc[j,'LDL-C']
                r_非hdl = temp_check_r.loc[j,'非HDL-C']

                time=temp_check_r.loc[j,'检验时间_date']
                df_drug_resuvatin_efficacy.loc[i,'第'+str(j+1)+'次血脂检查时间']=time
                
                if judge_efficacy(ascvd,r_ldl,r_非hdl,b_ldl,b_非hdl):
                    df_drug_resuvatin_efficacy.loc[i,'第'+str(j+1)+'次检查是否有效']=1
                else:
                    df_drug_resuvatin_efficacy.loc[i,'第'+str(j+1)+'次检查是否有效']=0

In [64]:
temp_check_r

,患者编号,检验时间_date,HDL-C,LDL-C,LPa,TC,TG,apo-A1,apo-B,非HDL-C
0,pat_0a31a5b06ad2a27c6d4dd950266235d9,2020-04-21,1.35,2.36,NaN,4.07,1.75,NaN,NaN,2.72


In [65]:
df_drug_resuvatin_efficacy.shape

(149, 38)

In [66]:
df_drug_resuvatin_efficacy.to_csv(project_path+'/data/processed_data/疗效/df_3.8_疗效评价.csv')